In [1]:
import pandas as pd
import warnings
warnings.filterwarnings("ignore")
import numpy as np

from category_encoders.target_encoder import TargetEncoder
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
import lightgbm as lgb

In [10]:
train = pd.read_csv('train_base.csv', sep=';')
test = pd.read_csv('test_base.csv', sep=';') 

In [7]:
from analytics import Analysis as als

In [11]:
bureau = pd.read_csv('bureau.csv')

In [13]:
list(bureau)

['SK_ID_CURR',
 'SK_ID_BUREAU',
 'CREDIT_ACTIVE',
 'CREDIT_CURRENCY',
 'DAYS_CREDIT',
 'CREDIT_DAY_OVERDUE',
 'DAYS_CREDIT_ENDDATE',
 'DAYS_ENDDATE_FACT',
 'AMT_CREDIT_MAX_OVERDUE',
 'CNT_CREDIT_PROLONG',
 'AMT_CREDIT_SUM',
 'AMT_CREDIT_SUM_DEBT',
 'AMT_CREDIT_SUM_LIMIT',
 'AMT_CREDIT_SUM_OVERDUE',
 'CREDIT_TYPE',
 'DAYS_CREDIT_UPDATE',
 'AMT_ANNUITY']

In [18]:
bureau

,SK_ID_CURR,SK_ID_BUREAU,CREDIT_ACTIVE,CREDIT_CURRENCY,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_LIMIT,AMT_CREDIT_SUM_OVERDUE,CREDIT_TYPE,DAYS_CREDIT_UPDATE,AMT_ANNUITY
0,215354,5714462,Closed,currency 1,-497,0,-153.0,-153.0,NaN,0,91323.00,0.0,NaN,0.0,Consumer credit,-131,NaN
1,215354,5714463,Active,currency 1,-208,0,1075.0,NaN,NaN,0,225000.00,171342.0,NaN,0.0,Credit card,-20,NaN
2,215354,5714464,Active,currency 1,-203,0,528.0,NaN,NaN,0,464323.50,NaN,NaN,0.0,Consumer credit,-16,NaN
3,215354,5714465,Active,currency 1,-203,0,NaN,NaN,NaN,0,90000.00,NaN,NaN,0.0,Credit card,-16,NaN
4,215354,5714466,Active,currency 1,-629,0,1197.0,NaN,77674.5,0,2700000.00,NaN,NaN,0.0,Consumer credit,-21,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1716423,259355,5057750,Active,currency 1,-44,0,-30.0,NaN,0.0,0,11250.00,11250.0,0.0,0.0,Microloan,-19,NaN
1716424,100044,5057754,Closed,currency 1,-2648,0,-2433.0,-2493.0,5476.5,0,38130.84,0.0,0.0,0.0,Consumer credit,-2493,NaN
1716425,100044,5057762,Closed,currency 1,-1809,0,-1628.0,-970.0,NaN,0,15570.00,NaN,NaN,0.0,Consumer credit,-967,NaN
1716426,246829,5057770,Closed,currency 1,-1878,0,-1513.0,-1513.0,NaN,0,36000.00,0.0,0.0,0.0,Consumer credit,-1508,NaN


In [24]:
bureau.shape

(1716428, 17)

In [23]:
als.check_missing_values(bureau, 10, 0)

AMT_ANNUITY               1226791
AMT_CREDIT_MAX_OVERDUE    1124488
DAYS_ENDDATE_FACT          633653
AMT_CREDIT_SUM_LIMIT       591780
AMT_CREDIT_SUM_DEBT        257669
DAYS_CREDIT_ENDDATE        105553
AMT_CREDIT_SUM                 13
CREDIT_TYPE                     0
AMT_CREDIT_SUM_OVERDUE          0
CNT_CREDIT_PROLONG              0
dtype: int64

In [19]:
als.get_unique_value(bureau, None)

SK_ID_CURR <class 'numpy.int64'> [215354 162297 402440 ... 448157 345866 235871]
SK_ID_BUREAU <class 'numpy.int64'> [5714462 5714463 5714464 ... 5057762 5057770 5057778]
CREDIT_ACTIVE <class 'str'> ['Closed' 'Active' 'Sold' 'Bad debt']
CREDIT_CURRENCY <class 'str'> ['currency 1' 'currency 2' 'currency 4' 'currency 3']
DAYS_CREDIT <class 'numpy.int64'> [-497 -208 -203 ...    0   -3   -1]
CREDIT_DAY_OVERDUE <class 'numpy.int64'> [   0 2603    6   30 2156  496  186 2264   41    8   71   13  252 1201
   12   38 1777  150   18   37   64 2050    9    4   29   22   49 1930
   34  951  936  664   45 2625  837   21   15   31 1715   19   67    7
   26   23   48   46   73   90 1127   53   33 1572   39 2582   14   25
   10   50   17  193   40   20   28  123  138  700  122 1504   32  670
   75  140   24  277 2019  161  962  187   58  106  634   83  120   60
   93 1694  227   44  344 2085 1248  124 1651 2541 1541  642  745  163
  234 1472  688 2455  614  319 2187   68  831  139  283  180   47  164
 

# Анализ данных

#### SK_ID_CURR - номер клиента


#### SK_ID_BUREAU - номер заявки


#### CREDIT_ACTIVE - статус кредита
##### Можно присвоить номера каждому статусу и искать сумму этого показателя. Например, чем больше итоговое число, тем больше вероятность, что кредит не вернут. 

#### CREDIT_CURRENCY - валюта кредита
##### Так как не стоит задавать порядок для разных валют, есть смысл сделать one-hot кодирование для это переменной. 

#### DAYS_CREDIT - 

#### CREDIT_DAY_OVERDUE - 

#### DAYS_CREDIT_ENDDATE - 

In [20]:
list(bureau)

['SK_ID_CURR',
 'SK_ID_BUREAU',
 'CREDIT_ACTIVE',
 'CREDIT_CURRENCY',
 'DAYS_CREDIT',
 'CREDIT_DAY_OVERDUE',
 'DAYS_CREDIT_ENDDATE',
 'DAYS_ENDDATE_FACT',
 'AMT_CREDIT_MAX_OVERDUE',
 'CNT_CREDIT_PROLONG',
 'AMT_CREDIT_SUM',
 'AMT_CREDIT_SUM_DEBT',
 'AMT_CREDIT_SUM_LIMIT',
 'AMT_CREDIT_SUM_OVERDUE',
 'CREDIT_TYPE',
 'DAYS_CREDIT_UPDATE',
 'AMT_ANNUITY']